In [2]:
import speech_recognition as sr

In [4]:
r = sr.Recognizer()
mic = sr.Microphone()

In [5]:
def callback(recognizer, audio):
        try:
          print(recognizer.recognize_google(audio))
        # handles any api/voice errors  errors
        except sr.RequestError:
          print( "There was an issue in handling the request, please try again")
        except sr.UnknownValueError:
          print("Unable to Recognize speech")

In [6]:
with mic as source:
    audio = r.listen_in_background(source, callback)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Abhishek\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Abhishek\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Abhishek\anaconda3\lib\site-packages\speech_recognition\__init__.py", line 690, in threaded_listen
    with source as s:
  File "C:\Users\Abhishek\anaconda3\lib\site-packages\speech_recognition\__init__.py", line 134, in __enter__
    assert self.stream is None, "This audio source is already inside a context manager"
AssertionError: This audio source is already inside a context manager


In [7]:
print("Starting speech recognition.")
# https://github.com/Uberi/speech_recognition/blob/master/reference/library-reference.rst
with sr.Microphone() as source:
    while True:
        try:
            audio = r.listen(source,phrase_time_limit=2) # listen to source
            # use testing api key
            text = r.recognize_google(audio, language="en-US")
            print("Users message: '{}'".format(text))
            if text == 'capture':
                print('saveHighlight')
        except sr.UnknownValueError:
            print("Sorry, I could not understand you.")
        except sr.RequestError:
            print("API call failed. Key valid? Internet connection?")

Starting speech recognition.
Users message: 'hello'
Users message: 'hello'
Users message: 'what is up'
Users message: 'that's cool'
Users message: 'let's go'
Users message: 'thank you'


KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2
from mss import mss
from PIL import ImageGrab


sct = mss()
img_np = []

out = cv2.VideoWriter('output.mp4', -1, 20.0, (640,480))

while 1:
    img = ImageGrab.grab(bbox=(0,0,960,540)) #bbox specifies specific region (bbox= x,y,width,height *starts top-left)
    img_np = np.array(img) #this is the array obtained from conversion
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    # cv2.imshow("test", frame)
    out.write(frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
import cv2
import numpy as np
import pyautogui

# display screen resolution, get it from your OS settings
SCREEN_SIZE = (1920, 1080)
# define the codec
fourcc = cv2.VideoWriter_fourcc(*"XVID")
# create the video write object
out = cv2.VideoWriter("output.avi", fourcc, 20.0, (1920,1080))

while True:
    # make a screenshot
    img = pyautogui.screenshot()
    # convert these pixels to a proper numpy array to work with OpenCV
    frame = np.array(img)
    # convert colors from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # write the frame
    out.write(frame)
    # show the frame
    cv2.imshow("screenshot", frame)
    # if the user clicks q, it exits
    if cv2.waitKey(1) == ord("q"):
        break

# make sure everything is closed when exited
out.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(0)
# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (640,  480))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv.flip(frame, 0)
    # write the flipped frame
    out.write(frame)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()

In [ ]:
import time

import cv2
import numpy as np
from mss import mss


def record(name):
    with mss() as sct:
        # mon = {'top': 160, 'left': 160, 'width': 200, 'height': 200}
        mon = sct.monitors[0]
        name = name + '.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        desired_fps = 30.0
        out = cv2.VideoWriter(name, fourcc, desired_fps,
                              (mon['width'], mon['height']))
        last_time = 0
        while True:
            img = sct.grab(mon)
            # cv2.imshow('test', np.array(img))
            if time.time() - last_time > 1./desired_fps:
                last_time = time.time()
                destRGB = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2BGR)
                out.write(destRGB)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break


record("Video")

In [11]:
import threading
import cv2
import numpy as np
import pyautogui
import speech_recognition as sr
import time


r = sr.Recognizer()
mic = sr.Microphone()

# display screen resolution, get it from your OS settings
SCREEN_SIZE = (1920, 1080)
# define the codec
fourcc = cv2.VideoWriter_fourcc(*"XVID")
# create the video write object
out = cv2.VideoWriter("highlight.avi", fourcc, 10.0, (1920,1080))

buffer = []

breakthread1 = 0

def maintain_buffer():
    while True:
        # make a screenshot
        img = pyautogui.screenshot()
        # convert these pixels to a proper numpy array to work with OpenCV
        frame = np.array(img)
        # convert colors from BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # write the frame
        if len(buffer) < 100:
            buffer.append(frame)
        else:
            buffer.pop(0)
            buffer.append(frame)
        if breakthread1 == 1:
            break

def capture_audio():
    print("Starting speech recognition.")
    # https://github.com/Uberi/speech_recognition/blob/master/reference/library-reference.rst
    with sr.Microphone() as source:
        while True:
            try:
                audio = r.listen(source,phrase_time_limit=2) # listen to source
                # use testing api key
                text = r.recognize_google(audio, language="en-US")
                print("Users message: '{}'".format(text))
                if text == 'capture':
                    print("Saving Highlight as requested")
                    breakthread1 = 1
                    balloon_tip('Highlight saved','Your highlight is being saved at C:/Users/abhishekl')
                    for i in range(len(buffer)):
                        out.write(buffer[i])
                    out.release()
                    break
            except sr.UnknownValueError:
                print("Sorry, I could not understand you.")
            except sr.RequestError:
                print("API call failed. Key valid? Internet connection?")
    
if __name__ == "__main__": 
    # creating thread
    t1 = threading.Thread(target=maintain_buffer) 
    t2 = threading.Thread(target=capture_audio) 
  
    # starting thread 1
    t1.start()
    # starting thread 2
    t2.start()
  
    # both threads completely executed 
    print("Listening!")

Starting speech recognition.Listening!

Sorry, I could not understand you.
Sorry, I could not understand you.
Sorry, I could not understand you.
Users message: 'hello'
Users message: 'what are you doing'
Users message: 'can you recognize me'
Users message: 'speed time'
Users message: 'so you do Miss words in between'
Users message: 'why is the m capital'
Users message: 'why are pretty accurate'
Users message: 'nevermind'
Users message: 'capture'
Saving Highlight as requested


In [ ]:
import notify2
ICON_PATH = "C:/Users/abhishekl/Desktop/Latest/GFN_logo1024.png"
notify2.init("Highlight Notifier")
n = notify2.Notification(None, icon = ICON_PATH)
n.set_urgency(notify2.URGENCY_CRITICAL)
n.set_timeout(5000)
n.update("Your highlight is saved successfully")
n.show()

In [8]:
# -- coding: utf-8 --
 
from win32api import *
from win32gui import *
import win32con
import sys, os
import struct
import time

class WindowsBalloonTip:
    def __init__(self, title, msg):
        message_map = {
                win32con.WM_DESTROY: self.OnDestroy,
        }
        # Register the Window class.
        wc = WNDCLASS()
        hinst = wc.hInstance = GetModuleHandle(None)
        wc.lpszClassName = "PythonTaskbar"
        wc.lpfnWndProc = message_map # could also specify a wndproc.
        classAtom = RegisterClass(wc)
        # Create the Window.
        style = win32con.WS_OVERLAPPED | win32con.WS_SYSMENU
        self.hwnd = CreateWindow( classAtom, "Taskbar", style, \
                0, 0, win32con.CW_USEDEFAULT, win32con.CW_USEDEFAULT, \
                0, 0, hinst, None)
        UpdateWindow(self.hwnd)
        iconPathName = os.path.abspath(os.path.join( sys.path[0], "balloontip.ico" ))
        icon_flags = win32con.LR_LOADFROMFILE | win32con.LR_DEFAULTSIZE
        try:
           hicon = LoadImage(hinst, iconPathName, \
                    win32con.IMAGE_ICON, 0, 0, icon_flags)
        except:
          hicon = LoadIcon(0, win32con.IDI_APPLICATION)
        flags = NIF_ICON | NIF_MESSAGE | NIF_TIP
        nid = (self.hwnd, 0, flags, win32con.WM_USER+20, hicon, "tooltip")
        Shell_NotifyIcon(NIM_ADD, nid)
        Shell_NotifyIcon(NIM_MODIFY, \
                         (self.hwnd, 0, NIF_INFO, win32con.WM_USER+20,\
                          hicon, "Balloon  tooltip",title,200,msg))
        # self.show_balloon(title, msg)
        time.sleep(10)
        DestroyWindow(self.hwnd)
    def OnDestroy(self, hwnd, msg, wparam, lparam):
        nid = (self.hwnd, 0)
        Shell_NotifyIcon(NIM_DELETE, nid)
        PostQuitMessage(0) # Terminate the app.
def balloon_tip(title, msg):
    w=WindowsBalloonTip(msg, title)

In [ ]:
time.sleep(10)